# <br>Analyzing and Visualizing Large Datasets

- Nov 14 2024 (continued from last week)

## We continue working with large datasets 

**By example:**
- Census data 
- NYC taxi cab trips
- Parking violations in NYC

### Fresh Environment (do in bash)

- You may have (every now and then) dependency conflicts due to incompatible package versions (same in R!!)
- Use a Fresh Virtual Environment: Creating a new environment is the safest way to avoid conflicts. If you're using conda, create an environment and install packages sequentially.
  - conda create -n myenv python=3.10
  - conda activate myenv

- Install Packages with Compatible Versions
  - pip install numpy==1.24.2 pandas==1.5.3 pip install bokeh==3.2.2 panel==1.0.4 holoviews==1.16.0 geoviews==1.13.0 pip install datashader==0.16.3

### Imports
- Import datashader and related modules:

In [ ]:
# some first class upgrades
#!pip install numpy --upgrade
#!pip install datashader --upgrade


# Initial imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import datashader as ds
import datashader.transfer_functions as tf
import zipfile
import requests
import dask.dataframe as dd
import fastparquet


import pyarrow.parquet as pq
import dask.dataframe as dd

# Color-related imports
from datashader.colors import Greys9, viridis, inferno
from colorcet import fire

# Ignore numpy warnings
np.seterr("ignore");

## Download and Extract: 

- First, download and unzip the file using Python's zipfile module or another tool.
- So here we work with the dataset locally (better if your internet is slow)
- Attention, that download can take time (3-4 minutes) - the census file is about 2GB
- Once extracted, Dask can read the actual parquet file.
- Then we read the parquet file and convert to a dask dataframe
- There are sometimes warnings, but don't worry about them for now

In [ ]:
# Download and extract
#url = "https://s3.amazonaws.com/datashader-data/census2010.parq.zip"
#response = requests.get(url)
#with open("census2010.parq.zip", "wb") as f:
#    f.write(response.content)

# print("ok")

# Extract the file
with zipfile.ZipFile("census2010.parq.zip", "r") as zip_ref:
    zip_ref.extractall("census_data")

# Read with pyarrow
table = pq.read_table("census_data/census2010.parq")

# Convert to a Pandas DataFrame
df = table.to_pandas()

# Then, convert to a Dask DataFrame
census_ddf = dd.from_pandas(df, npartitions=10)

Let's look into the structure of the dask dataframe

In [ ]:
census_ddf

In [ ]:
census_ddf.head()

## Plotting time!!!
Setup canvas parameters for USA image:

In [ ]:
from datashader.utils import lnglat_to_meters

In [ ]:
# Sensible lat/lng coordinates for U.S. cities
# NOTE: these are in lat/lng so EPSG=4326
USA = [(-124.72,  -66.95), (23.55, 50.06)]

# Get USA xlim and ylim in meters (EPSG=3857)
USA_xlim_meters, USA_ylim_meters = [list(r) for r in lnglat_to_meters(USA[0], USA[1])]

In [ ]:
# Define some a default plot width & height
plot_width  = 900
plot_height = int(plot_width*7.0/12)

Converting our ddf to a Panda DataFrame

In [ ]:
# Convert to a Pandas DataFrame
df = table.to_pandas()

# Convert the 'race' column to categorical if it’s not already
df['race'] = df['race'].astype('category')

Use a custom color scheme to map racial demographics:

In [ ]:
color_key = {"w": "aqua", "b": "lime", "a": "red", "h": "fuchsia", "o": "yellow"}

In [ ]:
def create_census_image(longitude_range, latitude_range, w=plot_width, h=plot_height):
    """
    A function for plotting the Census data, coloring pixel by race values.
    """
    # Step 1: Calculate x and y range from lng/lat ranges
    x_range, y_range = lnglat_to_meters(longitude_range, latitude_range)

    # Step 2: Setup the canvas
    canvas = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)

    # Step 3: Aggregate, but this time count the "race" category
    # NEW: specify the aggregation method to count the "race" values in each pixel
    agg = canvas.points(df, "easting", "northing", agg=ds.count_cat("race"))
    
    # Step 4: Shade, using our custom color map
    img = tf.shade(agg, color_key=color_key, how="eq_hist")

    # Return image with black background
    return tf.set_background(img, "black")

In [ ]:
img = create_census_image((-132.84029, -64.533035), (22.182915, 51.35972), w=1000, h=600)
img

## Can we learn more than just population density and race?

We can use *xarray* to slice the array of aggregated pixel values to examine specific aspects of the data.

### Question #1: Where do African Americans live?

Use the `sel()` function of the *xarray* array

In [ ]:
# Step 1: Setup canvas
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height)

census_ddf['race'] = census_ddf['race'].astype('category')
# Step 2: Aggregate and count race category
aggc = cvs.points(census_ddf, "easting", "northing", agg=ds.count_cat("race"))

In [ ]:
aggc

In [ ]:
# NEW: Select only African Americans (where "race" column is equal to "b")
agg_b = aggc.sel(race="b")

In [ ]:
agg_b

In [ ]:
# Step 3: Shade and set background
img = tf.shade(agg_b, cmap=fire, how="eq_hist")
img = tf.set_background(img, "black")

img

### Question #2: How to identify diverse areas?

**Goal:** Select pixels where each race has a non-zero count.

In [ ]:
aggc.sel(race=["w", "b", "a", "h"]) > 0

In [ ]:
(aggc.sel(race=['w', 'b', 'a', 'h']) > 0).all(dim='race')

In [ ]:
# Do a "logical and" operation across the "race" dimension
# Pixels will be "True" if the pixel has a positive count for each race
diverse_selection = (aggc.sel(race=['w', 'b', 'a', 'h']) > 200).all(dim='race')

diverse_selection

In [ ]:
# Select the pixel values where our diverse selection criteria is True
agg2 = aggc.where(diverse_selection).fillna(0)

# and shade using our color key
img = tf.shade(agg2, color_key=color_key)
img = tf.set_background(img,"black")

img 

### Question #3: Where is African American population greater than the White population?


In [ ]:
# Select where the "b" race dimension is greater than the "w" race dimension
selection = aggc.sel(race='h') > 4*(aggc.sel(race='w'))

selection

In [ ]:
# Select based on the selection criteria
agg3 = aggc.where(selection).fillna(0)

img = tf.shade(agg3, color_key=color_key)
img = tf.set_background(img, "black")

img

## Now let's make it interactive!


**Let's use hvplot**

In [ ]:
# Initialize hvplot and dask
import hvplot.pandas
import hvplot.dask # NEW: dask works with hvplot too!

import holoviews as hv
import geoviews as gv

### Side note: persisting dask arrays in memory

To speed up interactive calculations, you can "persist" a dask array in memory (load the data fully into memory). You should have at least 16 GB of memory to avoid memory errors, though!

If not persisted, the data will be loaded on demand to avoid memory issues, which will slow the interactive nature of the plots down slightly.

In [ ]:
census_ddf

In [ ]:
census_ddf = census_ddf.persist()

Here we scaled our coordinates, so we only focus on that area

In [ ]:
scaled_coordinatesX = (-13983766.89173708, -6883766.89173708)
scaled_coordinatesY = (2552839.908609666, 6256673.275328225)

In [ ]:
import hvplot.dask
import geoviews as gv
from holoviews import opts

# Ensure correct coordinate boundaries in meters for EPSG:3857
# Using previously defined USA_xlim_meters and USA_ylim_meters

# Plot the points using hvplot with datashader
points = census_ddf.hvplot.points(
    x="easting",
    y="northing",
    datashade=True,
    aggregator=ds.count(),
    cmap=fire,
    geo=True,
    crs="EPSG:3857",  # Specify that the input data is in EPSG:3857
    frame_width=plot_width,
    frame_height=plot_height,
    xlim=scaled_coordinatesX ,  # Set x bounds in meters
    ylim=scaled_coordinatesY,  # Set y bounds in meters
)

# Define the background tile source without specifying `crs`
bg = gv.tile_sources.CartoDark.options(width=plot_width, height=plot_height)

# Combine the tile source with the points plot
(bg * points).opts(
    opts.Tiles(xlim=scaled_coordinatesX , ylim=scaled_coordinatesY)  # Set x and y limits on the tile source as well
)


**Note:** interactive features (panning, zooming, etc) can be *slow*, but the map will eventually re-load!

### We can visualize color-coded race interactively as well

- Similar syntax to previous examples...
- Note how the `cmap` changed.

In [ ]:
# Points with categorical colormap
race_map = census_ddf.hvplot.points(
    x="easting",
    y="northing",
    datashade=True,
    c="race",  # NEW: color pixels by "race" column
    aggregator=ds.count_cat("race"),  # NEW: specify the aggregator
    cmap=color_key,  # NEW: use our custom color map dictionary
    crs=3857,
    geo=True,
    frame_width=plot_width,
    frame_height=plot_height,
    xlim=scaled_coordinatesX,
    ylim=scaled_coordinatesY,
)

bg = gv.tile_sources.CartoDark

bg * race_map

## Use case: exploring gerrymandering

- When we use examples of population, together with race or other demographics....
- Are some districts are drawn according to race distribution?
- We can easily overlay Congressional districts on our map...

In [ ]:
# Load congressional districts and convert to EPSG=3857
districts = gpd.read_file('./data/cb_2015_us_cd114_5m').to_crs(epsg=3857)

(let's check if the districts are correctly mapped), and if necessary, just plot the ones for the US.

In [ ]:
import hvplot.pandas  # This enables hvplot for both pandas and geopandas

districts_map = districts.hvplot.polygons(
    geo=True,
    crs="EPSG:3857",  # Ensure the CRS matches the plot
    line_color="white",
    fill_alpha=0.5,
    title="District Map"
)

districts_map

In [ ]:
# Plot the district map
districts_map = districts.hvplot.polygons(
    geo=True,
    crs=3857,
    line_color="white",
    fill_alpha=0,
    frame_width=plot_width,
    frame_height=plot_height,
    xlim=scaled_coordinatesX,
    ylim=scaled_coordinatesY
    
)

bg * districts_map

In [ ]:
race_map = census_ddf.hvplot.points(
    x="easting",
    y="northing",
    datashade=True,
    c="race",
    aggregator=ds.count_cat("race"),
    cmap=color_key,
    geo=True,
    crs="EPSG:3857",  # Explicitly specify the CRS
    frame_width=plot_width,
    frame_height=plot_height,
    xlim=USA_xlim_meters,
    ylim=USA_ylim_meters,
)

# Ensure the background and district maps are in EPSG:3857 as well
bg = gv.tile_sources.CartoDark.opts(xlim=scaled_coordinatesX, ylim=scaled_coordinatesY)
districts_map = districts_map.opts(xlim=scaled_coordinatesX, ylim=scaled_coordinatesY)

# Combine the maps
img = bg * race_map * districts_map
img

# Example 3: NYC taxi data

12 million taxi trips from 2015...

In [ ]:
import requests

# Download the file
url = "https://s3.amazonaws.com/datashader-data/nyc_taxi_wide.parq"
response = requests.get(url)
with open("nyc_taxi_wide.parq", "wb") as f:
    f.write(response.content)

In [ ]:
taxi_ddf = dd.read_parquet("nyc_taxi_wide.parq", engine="pyarrow", 
                               infer_divisions=True)

In [ ]:
taxi_ddf

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa

df = dd.read_parquet("nyc_taxi_wide.parq", engine="fastparquet")
    
# Save the dataframe without using RLE encoding
df.to_parquet("nyc_taxi_wide_reencoded.parq", engine="fastparquet", compression="SNAPPY", overwrite=True)


print("File successfully re-saved without RLE encoding.")

## Let's check our data 
- can you recognize the different columns, and what they mean?
- what would you be interested in mapping?
- could you map derived variables
  - trip length (by region, by time)
  - trip duration
  - origin - destination at different time of day
  - ....

In [ ]:
column_names = df.columns
print(column_names)

In [ ]:
print(f"{len(df)} Rows")
print(f"Columns: {list(df.columns)}")

### Trim your data
- What would you do if the data was extremely large?
- 64bit, 32bit??

In [ ]:
# Trim to the columns
df = df[
    [
        "passenger_count",
        "pickup_x",
        "pickup_y",
        "dropoff_x",
        "dropoff_y",
        "dropoff_hour",
        "pickup_hour",
    ]
]

In [ ]:
import dask.dataframe as dd
import hvplot.dask  # for Dask + hvPlot integration
import geoviews as gv
import hvplot.pandas  # for hvPlot functionality


# Create the map visualization
pickups_map = df.hvplot.points(
    x="pickup_x",
    y="pickup_y",
    color="blue",
    frame_width=800,
    frame_height=600,
    geo=True,
    crs="EPSG:3857",
    tiles="CartoDark"  # Adding tiles directly here for convenience
)

# Display the plot
pickups_map

### Exploring the taxi pick ups...

Group by the hour column to add a slider widget:

In [ ]:
pickups_map = df.hvplot.points(
    x="pickup_x",
    y="pickup_y",
    groupby="pickup_hour",
    cmap=fire,
    datashade=True,
    frame_width=800,
    frame_height=600,
    geo=True, 
    crs=3857
)

gv.tile_sources.CartoDark * pickups_map

### Comparing pickups and dropoffs

- Pixels with more pickups: *shaded red*
- Pixels with more dropoffs: *shaded blue*

In [ ]:
# Bounds in meters for NYC (EPSG=3857)
NYC = [(-8242000,-8210000), (4965000,4990000)]

# Set a plot width and height
plot_width  = int(750)
plot_height = int(plot_width//1.2)

In [ ]:
w = plot_width
h = plot_height

In [ ]:
x_range = NYC[0]
y_range = NYC[1]

In [ ]:
# Step 1: Create the canvas
cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)

# Step 2: Aggregate the pick ups
picks = cvs.points(df, "pickup_x", "pickup_y", ds.count("passenger_count"))

# Step 2: Aggregate the drop offs
drops = cvs.points(df, "dropoff_x", "dropoff_y", ds.count("passenger_count"))

# Rename to same names
drops = drops.rename({"dropoff_x": "x", "dropoff_y": "y"})
picks = picks.rename({"pickup_x": "x", "pickup_y": "y"})

In [ ]:
#picks

In [ ]:
#drops

In [ ]:
def create_merged_taxi_image(
    x_range, y_range, w=plot_width, h=plot_height, how="eq_hist"
):
    """
    Create a merged taxi image, showing areas with: 
    
    - More pickups than dropoffs in red
    - More dropoffs than pickups in blue
    """
    # Step 1: Create the canvas
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)

    # Step 2: Aggregate the pick ups
    picks = cvs.points(df, "pickup_x", "pickup_y", ds.count("passenger_count"))

    # Step 2: Aggregate the drop offs
    drops = cvs.points(df, "dropoff_x", "dropoff_y", ds.count("passenger_count"))

    # Rename to same names
    drops = drops.rename({"dropoff_x": "x", "dropoff_y": "y"})
    picks = picks.rename({"pickup_x": "x", "pickup_y": "y"})

    # Step 3: Shade
    # NEW: shade pixels there are more drop offs than pick ups
    # These are colored blue
    more_drops = tf.shade(
        drops.where(drops > picks), cmap=["darkblue", "cornflowerblue"], how=how
    )

    # Step 3: Shade
    # NEW: shade pixels where there are more pick ups than drop offs
    # These are colored red
    more_picks = tf.shade(
        picks.where(picks > drops), cmap=["darkred", "orangered"], how=how
    )

    # Step 4: Combine
    # NEW: add the images together!
    img = tf.stack(more_picks, more_drops)

    return tf.set_background(img, "black")

In [ ]:
LGA = [(-8228000, -8220000), (4977000, 4980000)]
create_merged_taxi_image(LGA[0], LGA[1])

In [ ]:
create_merged_taxi_image(NYC[0], NYC[1])

**Takeaway:** pickups occur more often on major roads, and dropoffs on smaller roads

## Let's generate a time-lapse GIF of drop-offs over time

Powerful tool for visualizing trends over time

### Define some functions...

**Important:** We can convert our datashaded images to the format of the Python Imaging Library (PIL) to visualize 

In [ ]:
def create_taxi_image(df, x_range, y_range, w=plot_width, h=plot_height, cmap=fire):
    """Create an image of taxi dropoffs, returning a Python Imaging Library (PIL) image."""
    
    # Step 1: Create the canvas
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    
    # Step 2: Aggregate the dropoff positions, coutning number of passengers
    agg = cvs.points(df, 'dropoff_x', 'dropoff_y',  ds.count('passenger_count'))
    
    # Step 3: Shade
    img = tf.shade(agg, cmap=cmap, how='eq_hist')
    
    # Set the background
    img = tf.set_background(img, "black")
    
    # NEW: return an PIL image
    return img.to_pil()

In [ ]:
def convert_to_12hour(hr24):
    """Convert from 24 hr to 12 hr."""
    from datetime import datetime
    d = datetime.strptime(str(hr24), "%H")
    return d.strftime("%I %p")

In [ ]:
def plot_dropoffs_by_hour(fig, data_all_hours, hour, x_range, y_range):
    """Plot the dropoffs for particular hour."""
    
    # Trim to the specific hour
    df_this_hour = data_all_hours.loc[data_all_hours["dropoff_hour"] == hour]

    # Create the datashaded image for this hour
    img = create_taxi_image(df_this_hour, x_range, y_range)

    # Plot the image on a matplotlib axes
    # Use imshow()
    plt.clf()
    ax = fig.gca()
    ax.imshow(img, extent=[x_range[0], x_range[1], y_range[0], y_range[1]])
    
    # Format the axis and figure
    ax.set_aspect("equal")
    ax.set_axis_off()
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    # Optional: Add a text label for the hour
    ax.text(
        0.05,
        0.9,
        convert_to_12hour(hour),
        color="white",
        fontsize=40,
        ha="left",
        transform=ax.transAxes,
    )

    # Draw the figure and return the image
    # This converts our matplotlib Figure into a format readable by imageio
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype="uint8")
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return image

### Strategy:

1. Create a datashaded image for each hour of taxi dropoffs, return as a PIL image object
1. Use matplotlib's `imshow()` to plot each datashaded image to a matplotlib Figure
1. Return each matplotlib Figure in a format readable by the `imageio` library 
1. Combine all of our images for each hours into a GIF using the `imageio` library

In [ ]:
import imageio

In [ ]:
# Create a figure
fig, ax = plt.subplots(figsize=(10, 10), facecolor="black")

# Create an image for each hour
imgs = []
for hour in range(24):

    # Plot the datashaded image for this specific hour
    print(hour)
    img = plot_dropoffs_by_hour(fig, taxi_ddf, hour, x_range=NYC[0], y_range=NYC[1])
    imgs.append(img)


# Combing the images for each hour into a single GIF
imageio.mimsave("dropoffsB.gif", imgs, duration=1000);

## Interesting aside: Beyond hvplot

**Analyzing hourly and weekly trends for taxis using holoviews**

- We'll load taxi data from 2016 that includes the number of pickups per hour.
- Visualize weekly and hourly trends using a *radial heatmap*

In [ ]:
df = pd.read_csv('./data/nyc_taxi_2016_by_hour.csv.gz', parse_dates=['Pickup_date'])

In [ ]:
df.head()

### Add date-related columns with `strftime`

We can use the `strftime()` function of a datetime Series to extract specific aspects of a date object. In this case, we are interested in:

- Day of Week (Monday, Tuesday, etc) and hour of day
- Week of Year

To find the specific string notation for these, use: http://strftime.org

In [ ]:
# create relevant time columns
df["Day & Hour"] = df["Pickup_date"].dt.strftime("%A %H:00")
df["Week of Year"] = df["Pickup_date"].dt.strftime("Week %W")
df["Date"] = df["Pickup_date"].dt.strftime("%Y-%m-%d")

In [ ]:
df.head()

### Let's plot a radial heatmap

The binning dimensions of the heat map will be:

- Day of Week and Hour of Day
- Week of Year

A radial heatmap can be read similar to tree rings:

- The center of the heatmap will represent the first week of the year, while the outer edge is the last week of the year
- Rotating clockwise along a specific ring tells you the day/hour.

In [ ]:
import holoviews as hv
hv.extension('bokeh')  # Ensure that you have loaded the holoviews extension for plotting

# Define the columns you want to use
cols = ["Day & Hour", "Week of Year", "Pickup_Count", "Date"]

# Create the heatmap
heatmap = hv.HeatMap(
    df[cols], kdims=["Day & Hour", "Week of Year"], vdims=["Pickup_Count", "Date"]
)

# Customize heatmap options (example adjustments)
heatmap.opts(
    width=600, height=400,  # Set plot dimensions
    colorbar=True,          # Add a colorbar for clarity
    cmap="Viridis",         # Choose a color map, e.g., 'Viridis'
    tools=['hover'],        # Enable hover tool for interactivity
    xlabel='Day & Hour',
    ylabel='Week of Year',
    title='Heatmap of Pickup Counts by Day & Hour and Week of Year'
)

# Display the heatmap
heatmap

In [ ]:
heatmap.opts(
    radial=True,
    height=600,
    width=600,
    yticks=None,
    xmarks=7,
    ymarks=3,
    start_angle=np.pi * 19 / 14,
    xticks=(
        "Friday",
        "Saturday",
        "Sunday",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
    ),
    tools=["hover"],
    cmap="fire"
)

### Trends

- Taxi pickup counts are high between 7-9am and 5-10pm during weekdays which business hours as expected. In contrast, during weekends, there is not much going on until 11am.
- **Friday and Saterday nights clearly stand out with the highest pickup densities** as expected.
- Public holidays can be easily identified. For example, taxi pickup counts are comparetively low around Christmas and Thanksgiving.
- Weather phenomena also influence taxi service. There is a very dark stripe at the beginning of the year starting at Saturday 23rd and lasting until Sunday 24th. Interestingly, there was one of the biggest blizzards in the history of NYC.

### Useful reference: the Holoviews example gallery

This radial heatmap example, and many more examples beyond hvplot available:

[https://holoviews.org/gallery/index.html](https://holoviews.org/gallery/index.html)

## Exercise: Datashading Philly parking violations data

#### Download the data

- A (large) CSV of parking violation data is available for download at: [https://musa550.s3.amazonaws.com/parking_violations.csv](https://musa550.s3.amazonaws.com/parking_violations.csv)
- Navigate to your browser, plug in the above URL, and download the data
- The data is from Open Data Philly: [https://www.opendataphilly.org/dataset/parking-violations](https://www.opendataphilly.org/dataset/parking-violations)
- Input data is in EPSG=4326
- **Remember**: You will need to convert latitude/longitude to Web Mercator (epsg=3857) to work with datashader.

### Step 1: Use `dask` to load the data

- The `dask.dataframe` module includes a [`read_csv()`](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv) function just like pandas 
- You'll want to specify the `assume_missing=True` keyword for that function: that will let dask know that some columns are allowed to have missing values

In [ ]:
import dask.dataframe as dd

In [ ]:
# I downloaded the data and moved it to the "data/" folder 
df = dd.read_csv("data/parking_violations.csv", assume_missing=True)

In [ ]:
df

In [ ]:
len(df)

In [ ]:
df.head()

### Step 2: Remove any rows with missing geometries

Remove rows that have NaN for either the `lat` or `lon` columns (*hint*: use the dropna() function!)

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
len(df)

### Step 3: Convert lat/lng to Web Mercator coordinates (x, y)

Add two new columns, `x` and `y`, that represent the coordinates in the EPSG=3857 CRS.

**Hint:** Use datashader's `lnglat_to_meters()` function.

In [ ]:
from datashader.utils import lnglat_to_meters

In [ ]:
# Do the conversion
x, y = lnglat_to_meters(df['lon'], df['lat'])

In [ ]:
# Add as columns
df['x'] = x
df['y'] = y

In [ ]:
df.head()

In [ ]:
df

### Step 4: Get the x/y range for Philadelphia for our canvas

- Convert the lat/lng bounding box into Web Mercator EPSG=3857 
- Use the `lnglat_to_meters()` function to do the conversion
- You should have two variables `x_range` and `y_range` that give you the corresponding `x` and `y` bounds 

In [ ]:
# Use lat/lng bounds for Philly
# This will exclude any points that fall outside this region
PhillyBounds = [( -75.28,  -74.96), (39.86, 40.14)]

In [ ]:
PhillyBoundsLng = PhillyBounds[0]
PhillyBoundsLat = PhillyBounds[1]

In [ ]:
# Convert to an EPSG=3857
x_range, y_range = lnglat_to_meters(PhillyBoundsLng, PhillyBoundsLat)

x_range

In [ ]:
# Optional: convert to lists as opposed to arrays
x_range = list(x_range)
y_range = list(y_range)

In [ ]:
x_range

### Step 5: Datashade the dataset
 
Create a matplotlib figure with the datashaded image of the parking violation dataset.

In [ ]:
# STEP 1: Create the canvas
cvs = ds.Canvas(plot_width=600, plot_height=600, x_range=x_range, y_range=y_range)

# STEP 2: Aggregate the points
agg = cvs.points(df, "x", "y", agg=ds.count())

# STEP 3: Shade the aggregated pixels
img = tf.shade(agg, cmap=fire, how="eq_hist")

# Optional: Set the background of the image
img = tf.set_background(img, "black")

# Show!
img

In [ ]:
type(img)

Let's add the city limits.

You'll need to convert your datashaded image to PIL format and use the `imshow()` function.

In [ ]:
# Load
city_limits = gpd.read_file("./data/City_Limits.geojson")

# Same CRS!
city_limits = city_limits.to_crs(epsg=3857)

In [ ]:
# Show with matplotlib
fig, ax = plt.subplots(figsize=(6, 6))

# NEW:
ax.imshow(img.to_pil(), extent=[x_range[0], x_range[1], y_range[0], y_range[1]])

# Format
ax.set_aspect("equal")
ax.set_axis_off()

# Add the city limits on top!
city_limits.plot(ax=ax, facecolor="none", edgecolor="white", linewidth=2)

### Step 6: Make an interactive map

Use hvplot to make an interactive version of your datashaded image.

In [ ]:
violations = df.hvplot.points(
    x="x",
    y="y",
    datashade=True,
    geo=True,
    crs=3857,
    frame_width=600,
    frame_height=600,
    cmap=fire,
    xlim=x_range,
    ylim=y_range,
)


gv.tile_sources.CartoDark * violations